In [1]:
import os, gc, time
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib as mpl
username = os.getenv('USER')
import dask.array as da
# from dask_jobqueue import SLURMCluster
# from dask.distributed import Client, progress
import dask
from collections import OrderedDict
dask.config.set(**{'array.slicing.split_large_chunks': False})
# dask.config.set({"distributed.workers.memory.target": 4})

from dscim import ProWaiter
from itertools import product, chain
import itertools
# import seaborn as sns
from functools import reduce
from pathlib import Path

import seaborn as sns


/home/kschwarz/miniconda3/envs/dscim2/lib/python3.8/site-packages/statsmodels/compat/pandas.py:61: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [2]:
discount_types = [ "euler_gwr", "euler_ramsey"] #, "constant",  "naive_gwr", "naive_ramsey"]
menu_options = ["risk_aversion", "adding_up", "equity"]
pulse_years = [2020]
# pulse_years = list(range(2025, 2051, 5))
weitzman_values = [x / 10.0 for x in range(1, 11, 1)] + [0.25, 0.01, 0.001, 0.0001]
global_cons = False
factors = False
marginal_damages = False
masks = [
    "unmasked",
        #  "truncate_at_ecs995symmetric_passing_mask",
        # "truncate_at_ecs990symmetric_passing_mask",
        # "truncate_at_ecs950symmetric_passing_mask",
        # "truncate_at_ecs830symmetric_passing_mask",
        # "truncate_at_ecs750symmetric_passing_mask",
        ]

fair_dims_list=[
    ['simulation'],
    # ['simulation', 'rcp'],
    # ['simulation', 'ssp', 'model'],
    # ['simulation', 'ssp', 'model', 'rcp'],
]

# run all

In [3]:
sectors = ["integration_AMEL"]
AR_list = [6]
combos = [(s, p, d, m, ar, ma, f) 
          for p in pulse_years
          for s in sectors 
          for d in discount_types 
          for m in menu_options
          for ar in AR_list
          for ma in masks
          for f in fair_dims_list
         ]

In [4]:
n_combos = len(combos)

while n_combos > 0: 
    combo = combos[0]
    print("=========================================================")
    print(combo)
    sector = combo[0]
    pulse_year = combo[1]
    discount_type = combo[2]
    menu_option = combo[3]
    AR = combo[4]
    mask = combo[5]
    fair_dims = combo[6]

    if len(fair_dims) > 1:
        print(fair_dims)
        fair_folder = f"fair_collapsed_{'_'.join([i for i in fair_dims if i!='simulation'])}"
    else:
        fair_folder = ''

    w = ProWaiter(path_to_config=f'/home/{username}/repos/integration/configs/96k_sims_test.yaml')
    save_path=f'/mnt/CIL_integration/menu_results_96k_integration/{sector}/{pulse_year}/{mask}/{fair_folder}'

    if mask=='unmasked':
        mask = None

    kwargs = {'discounting_type' : discount_type,
              'sector': sector,
              'ce_path' : f'/shares/gcp/integration/CE_library_epa_vsl_bc39/{sector}/',
              'save_path' : save_path,
              'fair_dims' : fair_dims,
              'weitzman_parameter' : weitzman_values,
              'pulse_year' : pulse_year,
              'ecs_mask_name' : mask,
              'eta' : 2.0,
              'rho' : 0.0,
             }

    if "CAMEL" in sector:
        kwargs.update({
            'damage_function_path' : f'/mnt/CIL_integration/damage_function_library/damage_function_library_integration/{sector}'
        })

    menu_item = w.menu_factory(menu_key=menu_option,
                                           sector=sector,
                                           kwargs=kwargs
                                          )
    
    print(menu_item.formula)
    if "CAMEL" not in sector:
        menu_item.damage_function_fit
        
    menu_item.order_plate('scc')
        
    if global_cons == True:
        menu_item.global_consumption_no_pulse.to_netcdf(
            f"{save_path}/{menu_option}_{discount_type}_eta{menu_item.eta}_rho{menu_item.rho}_global_consumption_no_pulse.nc4")
        menu_item.global_consumption.to_netcdf(
            f"{save_path}/{menu_option}_{discount_type}_eta{menu_item.eta}_rho{menu_item.rho}_global_consumption.nc4")

    if marginal_damages==True:
        md = (menu_item.global_consumption_no_pulse - menu_item.global_consumption_pulse) * menu_item.climate.conversion
        md = md.rename('marginal_damages').to_dataset()
        for var in md.variables:
            md[var].encoding.clear()
        md.chunk({
            "discount_type":1,
             "weitzman_parameter":1,
             "ssp":1,
             "model":1,
             "gas":1,
             "year":10
        }).to_zarr(
            f'{save_path}/{menu_option}_{discount_type}_eta{menu_item.eta}_rho{menu_item.rho}_uncollapsed_marginal_damages.zarr',
            consolidated = True,
            mode = "w"
        )

    if factors==True:
        
        # holding population constant
        # from 2100 to 2300 with 2099 values
        pop = menu_item.collapsed_pop.sum("region")
        pop = pop.reindex(
            year=range(pop.year.min().values, menu_item.ext_end_year + 1), method="ffill"
        )
        
        df = menu_item.calculate_discount_factors(
            menu_item.global_consumption_no_pulse / pop
        ).to_dataset(name = "discount_factor")
        for var in df.variables:
            df[var].encoding.clear()
        df.chunk({
            "discount_type":1,
             "weitzman_parameter":1,
             "ssp":1,
             "model":1,
             "gas":1,
             "year":10
        }).to_zarr(
            f'{save_path}/{menu_option}_{discount_type}_eta{menu_item.eta}_rho{menu_item.rho}_uncollapsed_discount_factors.zarr',
            consolidated = True,
            mode = "w"
        )

    combos.remove(combo)
    n_combos = len(combos)
    print(f"remaining combos: {n_combos}")

('integration_AMEL', 2020, 'euler_gwr', 'risk_aversion', 6, 'unmasked', ['simulation'])
damages ~ -1 + anomaly + np.power(anomaly, 2)
Risk-aversion CEs found at /shares/gcp/integration/CE_library_epa_vsl_bc39/integration_AMEL//risk_aversion_ce_no_cc_eta2.0.zarr. These are being loaded...
Subsetting on {'ssp': ['SSP2', 'SSP3', 'SSP4']} keys.
Risk-aversion CEs found at /shares/gcp/integration/CE_library_epa_vsl_bc39/integration_AMEL//risk_aversion_ce_cc_eta2.0.zarr. These are being loaded...
Subsetting on {'ssp': ['SSP2', 'SSP3', 'SSP4']} keys.
Subsetting on {'ssp': ['SSP2', 'SSP3', 'SSP4']} keys.


/home/kschwarz/miniconda3/envs/dscim2/lib/python3.8/site-packages/xarray/core/nanops.py:141: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


Subsetting on {'ssp': ['SSP2', 'SSP3', 'SSP4']} keys.
Saving /mnt/CIL_integration/menu_results_96k_integration/integration_AMEL/2020/unmasked/risk_aversion_euler_gwr_eta2.0_rho0.0_damage_function_points.csv
Extrapolating global consumption.
End-of-century growth rates are not capped.


/home/kschwarz/miniconda3/envs/dscim2/lib/python3.8/site-packages/xarray/core/nanops.py:141: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/kschwarz/miniconda3/envs/dscim2/lib/python3.8/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
/home/kschwarz/miniconda3/envs/dscim2/lib/python3.8/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))


Skipped saving global_consumption.
Saving /mnt/CIL_integration/menu_results_96k_integration/integration_AMEL/2020/unmasked/risk_aversion_euler_gwr_eta2.0_rho0.0_damage_function_fit.nc4


...

Saving /mnt/CIL_integration/menu_results_96k_integration/integration_AMEL/2020/unmasked/equity_euler_ramsey_eta2.0_rho0.0_uncollapsed_sccs.nc4


/home/kschwarz/miniconda3/envs/dscim2/lib/python3.8/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
/home/kschwarz/miniconda3/envs/dscim2/lib/python3.8/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
/home/kschwarz/miniconda3/envs/dscim2/lib/python3.8/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
/home/kschwarz/miniconda3/envs/dscim2/lib/python3.8/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
/home/kschwarz/miniconda3/envs/dscim2/lib/python3.8/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
/home/kschwarz/

Saving /mnt/CIL_integration/menu_results_96k_integration/integration_AMEL/2020/unmasked/equity_euler_ramsey_eta2.0_rho0.0_scc.nc4


/home/kschwarz/miniconda3/envs/dscim2/lib/python3.8/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
/home/kschwarz/miniconda3/envs/dscim2/lib/python3.8/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
/home/kschwarz/miniconda3/envs/dscim2/lib/python3.8/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
/home/kschwarz/miniconda3/envs/dscim2/lib/python3.8/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
/home/kschwarz/miniconda3/envs/dscim2/lib/python3.8/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
/home/kschwarz/

Results available: /mnt/CIL_integration/menu_results_96k_integration/integration_AMEL/2020/unmasked/
remaining combos: 0
